In [1]:
import pandas as pd
import torch
import os

from torch.utils.data import Dataset
from transformers import (BertTokenizer, BertForSequenceClassification, Trainer,
                          TrainingArguments, BertPreTrainedModel)
# from simpletransformers.language_modeling import LanguageModelingModel
from sklearn.metrics import accuracy_score, f1_score


In [2]:
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
print(torch.cuda.get_device_properties(0))

True
NVIDIA GeForce RTX 3060
_CudaDeviceProperties(name='NVIDIA GeForce RTX 3060', major=8, minor=6, total_memory=12050MB, multi_processor_count=28)


In [3]:
MODEL_NAME = '/home/abdullah/Code/dl/lt_bert/best_models/best_tiny_bert'
TRAIN_FILE_LOC = '/home/abdullah/Code/dl/bnlp-resources/sentiment/ABSA_datasets/BASA_cricket_train.tsv'
TEST_FILE_LOC = '/home/abdullah/Code/dl/bnlp-resources/sentiment/ABSA_datasets/BASA_cricket_test.tsv'
EVAL_FILE_LOC = '/home/abdullah/Code/dl/bnlp-resources/sentiment/ABSA_datasets/BASA_cricket_dev.tsv'


In [4]:
def tsv_to_text(tsv_file_loc):
    file_name = tsv_file_loc.split("/")[-1].split(".")[0]
    txt_name = tsv_file_loc.replace(".tsv", ".txt")
    txt_name = txt_name.replace("split_merged", "texts")

    if os.path.exists(txt_name):
        return [txt_name, file_name]

    df = pd.read_csv(tsv_file_loc, sep="\t")

    for txt in df["text"]:
        with open(txt_name, "a", encoding="utf8") as f:
            f.writelines(txt + "\n")
    return [txt_name, file_name]


tsv_to_text("/home/abdullah/Code/dl/bnlp-resources/sentiment/ABSA_datasets/BASA_cricket_dev.tsv")


['/home/abdullah/Code/dl/bnlp-resources/sentiment/ABSA_datasets/BASA_cricket_dev.txt',
 'BASA_cricket_dev']

In [5]:
def tsv_to_df(csv_file_loc):
    df = pd.read_csv(csv_file_loc, sep='\t')
    # remove id column
    df = df.drop(columns=['id'])

    # replace neutral with 0, positive with 1, negative with 2 in class_label column
    df.loc[df['class_label'] == 'neutral', 'class_label'] = 0
    df.loc[df['class_label'] == 'positive', 'class_label'] = 1
    df.loc[df['class_label'] == 'negative', 'class_label'] = 2

    return df


train_df = tsv_to_df(
    "/home/abdullah/Code/dl/bnlp-resources/sentiment/ABSA_datasets/BASA_restaurant_train.tsv")

texts = train_df['text'].tolist()
print(train_df)


                                                   text class_label
0     তাই, ভাল খাবারের জন্য আমিসুপারিশ করতাম, কিন্তু...           1
1     খাঁটি খাবার এবং তাদের কাছে সত্যিকারের সেবা আছে...           1
2     আমি  পাচ বছরেরও বেশি সময় ধরে উপরের হার্টল্যান...           1
3                                 এটা সুপারিশ আমি করেছি           1
4     জানিনা পূর্ববর্তী সমালোচক কে ছিলেন, লং, ডাইন্ড...           1
...                                                 ...         ...
1360  কিন্তু একরকম একটু অপেক্ষা করতে পারতাম,আমরা উদ্...           1
1361              গ্রেট bagels পুরানো-ভাবে তৈরির উপায় ।           1
1362     অ্যাট্রাকিয়াস থাকার কারনে খাবারটি ভয়াবহ ছিল।           2
1363  ওয়েটার আমাদের উপেক্ষা করেছিলেন তা আমরা আমাদের...           2
1364                            এই জায়গাটি ভীশন খারাপ।           2

[1365 rows x 2 columns]


In [6]:
def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  # calculate accuracy using sklearn's function
  acc = accuracy_score(labels, preds)
  return {
      'accuracy': acc,
  }


In [7]:
def f1_calculator(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  # calculate accuracy using sklearn's function
  f1 = f1_score(labels, preds, average='weighted')
  return {
      'f1': f1,
  }


In [8]:
train_list = tsv_to_text(TRAIN_FILE_LOC)
test_list = tsv_to_text(TEST_FILE_LOC)
eval_list = tsv_to_text(EVAL_FILE_LOC)


In [9]:
os.environ["WANDB_DISABLED"] = "true"

train_df = tsv_to_df(TRAIN_FILE_LOC)
test_df = tsv_to_df(TEST_FILE_LOC)
eval_df = tsv_to_df(EVAL_FILE_LOC)


class TINYDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(value[idx])
                for key, value in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)

train_encodings = tokenizer(train_df['text'].tolist(
), padding=True, truncation=True, max_length=512)
test_encodings = tokenizer(test_df['text'].tolist(
), padding=True, truncation=True, max_length=512)
eval_encodings = tokenizer(eval_df['text'].tolist(
), padding=True, truncation=True, max_length=512)

train_dataset = TINYDataset(train_encodings, train_df['class_label'].tolist())
test_dataset = TINYDataset(test_encodings, test_df['class_label'].tolist())
eval_dataset = TINYDataset(eval_encodings, eval_df['class_label'].tolist())

model = BertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=3).to('cuda')
model.manual_seed = 14


Some weights of the model checkpoint at /home/abdullah/Code/dl/lt_bert/best_models/best_tiny_bert were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model c

## 1 Epoch Train

In [10]:
# training_args = TrainingArguments(
#     output_dir=f"temp",
#     num_train_epochs=1,
#     per_device_train_batch_size=48,
#     per_device_eval_batch_size=96,
#     warmup_steps=500,
#     learning_rate=5e-5,
#     weight_decay=0.01,
#     overwrite_output_dir=True,
#     logging_dir=f"temp/logs",
#     logging_steps=44,
#     save_steps=44,
#     load_best_model_at_end=True,
#     evaluation_strategy="steps",
#     seed=14,
# )

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=eval_dataset

# )

# trainer.train()


Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
***** Running training *****
  Num examples = 2085
  Num Epochs = 1
  Instantaneous batch size per device = 48
  Total train batch size (w. parallel, distributed & accumulation) = 48
  Gradient Accumulation steps = 1
  Total optimization steps = 44
100%|██████████| 44/44 [00:01<00:00, 48.68it/s]***** Running Evaluation *****
  Num examples = 372
  Batch size = 96


{'loss': 1.1086, 'learning_rate': 4.4e-06, 'epoch': 1.0}


                                               
100%|██████████| 44/44 [00:01<00:00, 48.68it/s]Saving model checkpoint to temp/checkpoint-44
Configuration saved in temp/checkpoint-44/config.json


{'eval_loss': 1.0908746719360352, 'eval_runtime': 0.0396, 'eval_samples_per_second': 9390.798, 'eval_steps_per_second': 100.976, 'epoch': 1.0}


Model weights saved in temp/checkpoint-44/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from temp/checkpoint-44 (score: 1.0908746719360352).
100%|██████████| 44/44 [00:02<00:00, 18.29it/s]

{'train_runtime': 2.416, 'train_samples_per_second': 862.981, 'train_steps_per_second': 18.212, 'train_loss': 1.1085989691994407, 'epoch': 1.0}


TrainOutput(global_step=44, training_loss=1.1085989691994407, metrics={'train_runtime': 2.416, 'train_samples_per_second': 862.981, 'train_steps_per_second': 18.212, 'train_loss': 1.1085989691994407, 'epoch': 1.0})

## 100 Epoch with 1 epoch model

In [12]:
MODEL_NAME = '/home/abdullah/Code/dl/lt_bert/best_models/1/1_cricket_sentiment'
model = BertForSequenceClassification.from_pretrained(
    MODEL_NAME, num_labels=3).to('cuda')
model.manual_seed = 14

for name, param in model.named_parameters():
    if 'bert.encoder.layer.0' in name:
        # print(name, param.requires_grad)
        param.requires_grad = False
# print("\n\n\n")
# for name, param in model.named_parameters():
#     print(name, param.requires_grad)

training_args = TrainingArguments(
    output_dir=f"temp",
    num_train_epochs=100,
    per_device_train_batch_size=192,
    per_device_eval_batch_size=384,
    warmup_steps=500,
    learning_rate=5e-5,
    weight_decay=0.01,
    overwrite_output_dir=True,
    logging_dir=f"temp/logs",
    logging_steps=100,
    save_steps=100,
    load_best_model_at_end=True,
    evaluation_strategy="steps",
    seed=14,
    label_smoothing_factor=0.5,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=f1_calculator,

)

trainer.train()

# print(cool.metrics)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,

)

trainer.train()


loading configuration file /home/abdullah/Code/dl/lt_bert/best_models/1/1_cricket_sentiment/config.json
Model config BertConfig {
  "_name_or_path": "/home/abdullah/Code/dl/lt_bert/best_models/best_tiny_bert",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 128,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 512,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 2,
  "num_hidden_layers": 2,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.12.5",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size

{'loss': 1.0793, 'learning_rate': 1e-05, 'epoch': 9.09}


                                                  
  9%|▉         | 100/1100 [00:04<00:46, 21.62it/s]Saving model checkpoint to temp/checkpoint-100
Configuration saved in temp/checkpoint-100/config.json


{'eval_loss': 1.0488898754119873, 'eval_f1': 0.6248210659476015, 'eval_runtime': 0.0476, 'eval_samples_per_second': 7814.69, 'eval_steps_per_second': 21.007, 'epoch': 9.09}


Model weights saved in temp/checkpoint-100/pytorch_model.bin
 18%|█▊        | 200/1100 [00:10<00:41, 21.65it/s]***** Running Evaluation *****
  Num examples = 372
  Batch size = 384


...


100%|██████████| 1100/1100 [01:00<00:00, 21.17it/s]Saving model checkpoint to temp/checkpoint-1100
Configuration saved in temp/checkpoint-1100/config.json


{'eval_loss': 1.0100125074386597, 'eval_accuracy': 0.7473118279569892, 'eval_runtime': 0.0419, 'eval_samples_per_second': 8888.667, 'eval_steps_per_second': 23.894, 'epoch': 100.0}


Model weights saved in temp/checkpoint-1100/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from temp/checkpoint-100 (score: 1.0011996030807495).
100%|██████████| 1100/1100 [01:01<00:00, 17.78it/s]

{'train_runtime': 61.8619, 'train_samples_per_second': 3370.408, 'train_steps_per_second': 17.782, 'train_loss': 0.9781355701793324, 'epoch': 100.0}


TrainOutput(global_step=1100, training_loss=0.9781355701793324, metrics={'train_runtime': 61.8619, 'train_samples_per_second': 3370.408, 'train_steps_per_second': 17.782, 'train_loss': 0.9781355701793324, 'epoch': 100.0})